# OPTIMIZATION PHASES

### List of variables

<table>
    <thead>
        <tr>
            <th style="width: 10%">Variable</th>
            <th style="width: 45%">Description</th>
            <th style="width: 30%">Comment</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>$B$</td>
            <td>Number of full blocks/pages that need the records</td>
            <td>$\lceil \frac{|T|}{R} \rceil$; $B \ll |T|$</td>
        </tr>
        <tr>
            <td>$R$</td>
            <td>Number of records per block/page</td>
            <td></td>
        </tr>
        <tr>
            <td>$|T|$</td>
            <td>Cardinality. Number of tuples of a table</td>
            <td>Size of table</td>
        </tr>
        <tr>
            <td>$D$</td>
            <td>Time to access (read or write) a disk block</td>
            <td>Approximately 0'010 seconds</td>
        </tr>
        <tr>
            <td>$C$</td>
            <td>Time for the CPU to process a record</td>
            <td>Approximately 10<sup>-9</sup></td>
        </tr>
        <tr>
            <td>$d$</td>
            <td>Tree order</td>
            <td>Usually greater than 100</td>
        </tr>
        <tr>
            <td>$h$</td>
            <td>Tree depth minus 1</td>
            <td>$\lceil \log_u |T| \rceil - 1$</td>
        </tr>
        <tr>
            <td>$v$</td>
            <td>Number of different values in a search</td>
            <td></td>
        </tr>
        <tr>
            <td>$u$</td>
            <td>$\%load \cdot 2d$</td>
            <td></td>
        </tr>
        <tr>
            <td>$k$</td>
            <td>Number of repetitions of every value in the search</td>
            <td></td>
        </tr>
        <tr>
            <td>ndist(A)</td>
            <td>Number of different values for attribute A</td>
            <td>Obtained from DB catalog</td>
        </tr>
        <tr>
            <td>max</td>
            <td>Maximum value of an attribute</td>
            <td>Obtained from DB catalog</td>
        </tr>
        <tr>
            <td>min</td>
            <td>Minimum value of an attribute</td>
            <td>Obtained from DB catalog</td>
        </tr>
        <tr>
            <td>$H$</td>
            <td>Time to evaluate the hash function</td>
            <td></td>
        </tr>
        <tr>
            <td>$M$</td>
            <td>Memory pages for a join/sorting algorithm</td>
            <td></td>
        </tr>
        <tr>
            <td>bits</td>
            <td>Bits per index block</td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td>Domain Cardinality</td>
            <td>Maximum number of different values</td>
        </tr>
    </tbody>
</table>

### List of variables for intermediate results

Record length. $\sum$ attribute length<sub>i</sub> (+ control information)
$$|R|$$

Number of records per block
$$R_R = \lfloor \frac{B}{|R|} \rfloor$$

Number of blocks per table
$$B_R = \lceil \frac{|R|}{R_R} \rceil$$


###  Cardinalities estimation

Selectivity Factor. % of tuples in the output regarding the input. ~0: very selective ~1: not very selective
$$\mathrm{SF}$$

Output of cardinality estimation
$$|O| = \mathrm{SF} \cdot |R|$$ or $$|O| = \mathrm{SF} \cdot |R1| \cdot |R2|$$ 

Selection
$$|\mathrm{selection}(R)| = \mathrm{SF} \cdot |R|$$

Join
$$|\mathrm{join}(R, S)| = \mathrm{SF} \cdot |R| \cdot |S|$$

Unions with repetitions
$$|\mathrm{union}(R, S)| = |R| + |S|$$

Unions without repetitions
$$|\mathrm{union}(R, S)| = |R| + |S| - |\mathrm{union}(R, S)|$$

Difference (anti-join)
$$|\mathrm{difference}(R, S)| = |R| - |\mathrm{union}(R, S)|$$


## Optimization phases

**Asumptions**

* Materialized views
* Focus on Disk access time
* Physical address of the record
* Only consider cases
  1. No index
  2. Unordered B-Tree with addresses (B<sup>+</sup>)
  3. Unordered Hash with addresses
  4. Orderered B-Tree with addresses (Clustered)


### Unordered B-tree with addresses (B<sup>+</sup>)

**Assumptions**

* In every tree node **2d** addresses fit
* Tree load 66% (2/3)

### Orderered B-Tree with addresses (Clustered)

**Assumptions**

* Tree load 66% (2/3) (index and table blocks)

### Unordered Hash with addresses

**Assumptions**

* No blocks for excess
* The same number of entries fit in a bucket block as in a tree block
* Bucket blocks at 80% (4/5)

## Space


**No index**

$$B$$

**B<sup>+</sup>**

$$\sum_1^{h+1} \lceil \frac{|T|}{u^i} \rceil + B$$

**Clustered**

$$\sum_1^{h+1} \lceil \frac{|T|}{u^i} \rceil + \lceil 1.5B \rceil$$

**Hash**

$$1 + \lceil 1.25(\frac{|T|}{2d}) \rceil + B$$


Example:

$$\mathrm{Lvl_1} = \frac{|T|}{u}$$

$$\mathrm{Lvl_2} = \frac{|T|}{u^2}$$

$$\mathrm{Lvl_3} = \frac{|T|}{u^3}$$

## Access paths

### Table scan

The whole table

<div style="text-align: right"> $u = \frac{2}{3} \cdot 2d$ </div>

**No index**

$$B \cdot D$$

**B<sup>+</sup>**

<span style="color:orange">Only useful for sort</span>

$$\lceil \frac{|T|}{u} \rceil \cdot D + |T| \cdot D$$

**Clustered**

$$\lceil 1.5B \rceil \cdot D$$

**Hash**

<span style="color:red">Useless</span>

$$\lceil 1.25(\frac{|T|}{2d}) \rceil \cdot D + |T| \cdot D $$



### Search one tuple

Equality of unique attribute

<div style="text-align: right">
    $u = \frac{2}{3} \cdot 2d$ <br>
    $h = \lceil \log_u |T| \rceil - 1$
</div>

**No index**

$$0.5B \cdot D$$

**B<sup>+</sup>**

$$h \cdot D + D$$

**Clustered**

$$h \cdot D + D$$

**Hash**

$$H + D + D$$


### Search several tuples

Interval

No unique attribute

<div style="text-align: right">
    $u = \frac{2}{3} \cdot 2d$ <br>
    $h = \lceil \log_u |T| \rceil - 1$ <br>
    $|O|$: cardinality of Output <br>
    $v$: value in range <br>
    $k$: repetitions per value <br>
</div>

**No index**

$$B \cdot D$$

**B<sup>+</sup>**

$$h \cdot D + \frac{|O| - 1}{u} \cdot D + |O| \cdot D$$

**Clustered**

$$h \cdot D + D + 1.5 \left( \frac{|O|-1}{R} \right) \cdot D$$

**Hash**

$$v = 1: 1 \cdot (H + D + k \cdot D) = H + D + k \cdot D$$

$$v > 1: v \cdot (H + D + k \cdot D)$$

$$v \;\mathrm{is\;unknown}: \mathrm{Useless}$$



### Statistics in Oracle

DBA is responsible for the statistics.

`ANALYZE [TABLE|INDEX|CLUSTER] <name> [COMPUTE|ESTIMATE] STATISTICS;`

```sql
ANALYZE TABLE departments COMPUTE STATISTICS; 
ANALYZE TABLE employees COMPUTE STATISTICS;
```

`DBMS_STATS.GATHER_TABLE_STATS( <esquema>, <table> );`

```sql
DBMS_STATS.GATHER_TABLE_STATS("username", "departments");
DBMS_STATS.GATHER_TABLE_STATS("username", "employees");
```

Kinds of statistics

| Relations | Attributes |
|:--|:--|
| Cardinality | Length |
| Number of blocks | Domain cardinality |
| Average length of records | Number of existing different values |
| | Maximum value |
| | Minimum value |

Main hypothesis in most DBMS
* Uniform distribution of values for each attribute
* Independence of attributes

## Selectivity Factor of a Selection

Assuming equi-probability of values

`WHERE A = c`
$$\mathrm{SF}(A = c) = \frac{1}{\mathrm{ndist}(A)}$$

Assuming uniform distribution and $A \in [\min, \max]$ 

`WHERE A > c`

$$
\mathrm{SF}(A > c) = \frac{\max - c}{\max - \min} =
\begin{cases}
0 & \quad \text{if}\; c \geq \max \\
1 & \quad \text{if}\; c < \min
\end{cases}
$$

`WHERE A < c`

$$
\mathrm{SF}(A < c) = \frac{c - \min}{\max - \min} =
\begin{cases}
0 & \quad \text{if}\; c \leq \min \\
1 & \quad \text{if}\; c > \max
\end{cases}
$$


Assuming $\text{ndist}(A)$ is big enough

`WHERE A <= c`
$$
\mathrm{SF}(A \leq c) = \mathrm{SF}(A < c)
$$

`WHERE A >= c`
$$
\mathrm{SF}(A \geq c) = \mathrm{SF}(A > c)
$$

Assuming P and Q statistically **independent**

`WHERE P AND Q`
$$
\text{SF}(P \;\text{AND}\; Q) = \text{SF}(P) \cdot \text{SF}(Q)
$$

`WHERE P OR Q`
$$
\text{SF}(P \;\text{OR}\; Q) = \text{SF}(P) + \text{SF}(Q) - \text{SF}(P) \cdot \text{SF}(Q)
$$

`WHERE NOT P`
$$
\text{SF}(\text{NOT}\;P) = 1 - \text{SF}(P)
$$

`WHERE A IN (c1, c2, ... , cn)`
$$
\text{SF}(A \in (c_1, c_2, \dots, c_n)) = \min(1, \frac{n}{\mathrm{ndist}(A)})
$$

`WHERE A BETWEEN (c1, c2)`
$$
\text{SF}(c_1 \leq A \leq c_2) = \frac{\min(c_2, \max)-\max(c_1, \min)}{\max - \min}
$$

### Selectivity Factor of a Join

For $R[A \theta B]S$

Too difficult to approximate this general case. Usually, the required statistics are not available because it
would be too expensive to maintain them.

Results depend on operator:

$$
\text{SF}(R[A\times B]S) = 1
$$

$$
\text{SF}(R[A \neq B]S) = 1 
$$

$$
\text{SF}(R[A=B]S) = \frac{1}{|R|} 
\begin{cases}
S_B & \quad \text{is not null} \\
S_B & \quad \text{FK to } R_A \\
R_A & \quad \text{PK}
\end{cases}
$$

If there is no FK

$$
\text{SF}(R[A=B]S) = \frac{1}{\max(\text{ndist}(A), \text{ndist}(B))}
$$

$$
\text{SF}(R[A<B]S) = {^1/_2}
$$
$$
\text{SF}(R[A \leq B]S) = {^1/_2}
$$

## Phases of physiscal optimization

1. Alternatives generation
2. Intermediate results estimation
3. Cost estimation for each algorithm
4. Choose the best option

## Example

```sql
SELECT 
    DISTINCT w.strength
FROM
    wines w, producers p, vintages v
WHERE
    v.wineId = w.wineId
    AND
    p.prodId = v.prodId
    AND
    p.region = "Priorat"
    AND
    v.quantity > 100;
```

![processtree](processtree.svg)

Tables have the following structures

Producers
* Clustered by `prodId`
* B<sup>+</sup> by `region`

Wines
* Clustered by `wineId`

Vintages
* Clustered by `wineId` and `prodId`

Statistics:
Tables (extra space due to being clustered needs to be added)

$$
\begin{matrix}
|P| = 10000 & |W| = 5000 & |V| = 100000 \\
R_p = 12    & R_w = 10 & R_v = 20 \\
B_p = 834   & B_w = 500  & B_v = 5000
\end{matrix}
$$

Attributes

prodId, wineId and strength: $|R_R| = 5$ bytes

$\text{ndist(region)} = 30$

$\min(\text{quantity}) = 10$ $\max(\text{quantity}) = 500$

$\text{ndist(strength)} = 10$


**System Parameters**

$B = 500$ bytes per intermediate disk block

$D = 1$

$C = 0$

$d = 75$

DBMS:

* Block Nested Loops (6 Memory pages, $M = 4$)
* Row Nested Loops
* Sort Match (with 3 memory pages for sorting, $M = 2$)


In [9]:
import math

c_P, c_W, c_V = 10000, 5000, 100000
R_p, R_w, R_v = 12, 10, 20
B_p, B_w, B_v = math.ceil(c_P / R_p), math.ceil(c_W / R_w), math.ceil(c_V / R_v)

print("Cardinality of {}: {}, Records: {}, number of Full Blocks: {}".format('P', c_P, R_p, B_p))
print("Cardinality of {}: {}, Records: {}, number of Full Blocks: {}".format('W', c_W, R_w, B_w))
print("Cardinality of {}: {}, Records: {}, number of Full Blocks: {}".format('V', c_V, R_v, B_v))

Cardinality of P: 10000, Records: 12, number of Full Blocks: 834
Cardinality of W: 5000, Records: 10, number of Full Blocks: 500
Cardinality of V: 100000, Records: 20, number of Full Blocks: 5000


### Phase 1. Alternatives generation

```sql
SELECT 
    DISTINCT w.strength
FROM
    wines w, producers p, vintages v
WHERE
    v.wineId = w.wineId AND p.prodId = v.prodId 
    AND p.region = "Priorat"
    AND v.quantity > 100;
```

Change selection and join arrangement

![Phase 1](phase1.svg)

### Phase 2. Intermediate results estimation

```sql
SELECT 
    DISTINCT w.strength
FROM
    wines w, producers p, vintages v
WHERE
    v.wineId = w.wineId AND p.prodId = v.prodId 
    AND p.region = "Priorat"
    AND v.quantity > 100;
```

**PT1 and PT2**

**Selection over V: V'**

![V: V'](vvprime.svg)

Record length of prodId and wineId:
$$|R_{V'}| = 5 + 5 = 10$$

Selectivity factor of selection:
$$
\mathrm{SF}(A > c) = \frac{\max - c}{\max - \min}
$$

Where $c = 100$ and the query specifies `v.quantity > 100`, then:

$$
\text{SF}(\text{quantity} > 100) = \frac{500 - 100}{500 - 10} = 0.81632
$$

Output cardinality of V':
$$
|O| = \text{SF} \cdot |R|
$$

$$
|V'| = \text{SF}(\text{quantity} > 100) \cdot |V| = 0.81632 \cdot 100000 = 81632
$$

Number of records per block:
$$
R_{V'} = \lfloor \frac{B}{|R_{V'}|} \rfloor = \lfloor \frac{500}{10} \rfloor = 50
$$

Number of blocks needed for V':
$$
B_{V'} = \lceil \frac{|V'|}{R_{V'}} \rceil = \lceil \frac{81632}{50} \rceil = 1633
$$

In [2]:
c = 100
min_v = 10
max_v = 500
SF_v_prime = (max_v - c) / (max_v - min_v)
print("Selectivity factor of V': {} \n".format(SF_v_prime))

C_v_prime = math.floor(SF_v_prime * c_V)
print("Cardinality output of V': {} \n".format(C_v_prime))

R_v_len = 5 + 5
B = 500
R_v_prime = math.floor(B / R_v_len)
print("V' number of records per block : {} \n".format(R_v_prime))

B_v_prime = math.ceil(C_v_prime / R_v_prime)
print("Blocks needed for V': {} \n".format(B_v_prime))

Selectivity factor of V': 0.8163265306122449 

Cardinality output of V': 81632 

V' number of records per block : 50 

Blocks needed for V': 1633 



**Selection over P: P'**

![P: P'](ppprime.svg)

Record length of prodId:
$$|R_{P'}| = 5$$

Selectivity factor of selection:
$$
\mathrm{SF}(A = c) = \frac{1}{\text{ndist}(A)}
$$

Where $c = 'Priorat'$ and the query specifies `p.region = 'Priorat'`, then:

$$
\text{SF}(\text{region} = \text{Priorat}) = \frac{1}{30} = 0.033333
$$

Output cardinality of P':
$$
|O| = \text{SF} \cdot |R|
$$

$$
|P'| = \text{SF}(\text{region} = \text{Priorat}) \cdot |P| = 0.03333 \cdot 10000 = 333
$$

Number of records per block:
$$
R_{P'} = \lfloor \frac{B}{|R_{P'}|} \rfloor = \lfloor \frac{500}{5} \rfloor = 100
$$

Number of blocks needed for P':
$$
B_{P'} = \lceil \frac{|P'|}{R_{P'}} \rceil = \lceil \frac{333}{100} \rceil = 4
$$

In [3]:
ndist_region = 30
SF_p_prime = 1 / ndist_region
print("Selectivity factor of P': {} \n".format(SF_p_prime))

C_p_prime = math.floor(SF_p_prime * c_P)
print("Cardinality output of P': {} \n".format(C_p_prime))

R_p_len = 5
B = 500
R_p_prime = math.floor(B / R_p_len)
print("P' number of records per block : {} \n".format(R_p_prime))

B_p_prime = math.ceil(C_p_prime / R_p_prime)
print("Blocks needed for P': {} \n".format(B_p_prime))

Selectivity factor of P': 0.03333333333333333 

Cardinality output of P': 333 

P' number of records per block : 100 

Blocks needed for P': 4 



**PT1**

**Join between W and V': WV'**

![WV'](wvprime.svg)

Record length of `strength` and `prodId`:
$$
|R_{WV'}| = 5 + 5
$$

Selectivity factor

$$
\text{SF}_{WV'} = \frac{1}{|W|} = \frac{1}{5000} = 0.0002
$$

Cardinality ouput of WV'
$$
|WV'| = SF_{WV'} \cdot |W| \cdot |V'| = \frac{1}{5000} \cdot 5000 \cdot  81632 = 81632
$$

Number of rows per block for WV':
$$
R_{WV'} = \lfloor \frac{B}{|R_{WV'}|} \rfloor = \lfloor \frac{500}{5} \rfloor = 50
$$

Number of blocks used for WV':

$$
B_{WV'} = \lceil \frac{|WV'|}{R_{WV'}} \rceil = \lceil \frac{81632}{50} \rceil = 1633
$$

In [4]:
SF_wv_prime = 1 / c_W
print("Selectivity factor of WV': {} \n".format(SF_wv_prime))

C_wv_prime = math.floor(SF_wv_prime * c_W * C_v_prime)
print("Cardinality output of WV': {} \n".format(C_wv_prime))

R_wv_prime_len = 5 + 5
B = 500
R_wv_prime = math.floor(B / R_wv_prime_len)
print("WV' number of records per block : {} \n".format(R_wv_prime))

B_wv_prime = math.ceil(C_wv_prime / R_wv_prime)
print("Blocks needed for WV': {} \n".format(B_wv_prime))

Selectivity factor of WV': 0.0002 

Cardinality output of WV': 81632 

WV' number of records per block : 50 

Blocks needed for WV': 1633 



**Join between WV' and P': WV'P'**

![WV'P'](wvprimepprime.svg)

Record length for `strength`:
$$
|R_{WV'P'}| = 5
$$

Selectivity Factor, assuming quantity and region independent
$$
\text{SF(WV'} \cdot \text{P')} = \frac{1}{|P'|} \cdot \frac{1}{ndist(\text{region})} = \frac{1}{333 \cdot 30} = 10^{-4}
$$

Cardinality output
$$
|WV'P'| = SF_{WV'P'} \cdot |WV'| \cdot |P'| = 10^{-4} \cdot 81632 \cdot 333 = 2721
$$

Records per block
$$
R_{WV'P'} = \lfloor \frac{B}{|R_{WV'P'}|} \rfloor = \lfloor \frac{500}{5} \rfloor = 100
$$

Blocks for WV'P'
$$
B_{WV'P'} = \lceil \frac{|WV'P'|}{R_{WV'P'}} \rceil = \lceil \frac{1234}{100} \rceil = 28
$$

In [5]:
SF_wvp_prime = (1 / C_p_prime) * (1 / ndist_region)
print("Selectivity factor of WV'P': {} \n".format(SF_wvp_prime))

C_wvp_prime = math.floor(SF_wvp_prime * C_wv_prime * C_p_prime)
print("Cardinality output of WV'P': {} \n".format(C_wvp_prime))

R_wvp_prime_len = 5
B = 500
R_wvp_prime = math.floor(B / R_wvp_prime_len)
print("WV'P' number of records per block : {} \n".format(R_wvp_prime))

B_wvp_prime = math.ceil(C_wvp_prime / R_wvp_prime)
print("Blocks needed for WV'P': {} \n".format(B_wvp_prime))

Selectivity factor of WV'P': 0.00010010010010010009 

Cardinality output of WV'P': 2721 

WV'P' number of records per block : 100 

Blocks needed for WV'P': 28 



**PT2**

![V'P'](vprimepprime.svg)

**Join V' and P': V'P'**

Assuming independence of variables

Record length for `wineId`
$$
|R_{V'P'}| = 5
$$

Selectivity factor
$$
\text{SF}_{V'P'} = \frac{1}{ndist(\text{region})} \cdot \frac{1}{|P'|} = \frac{1}{30} \cdot \frac{1}{333} = 10^{-4}
$$

Output cardinality
$$
|V'P'| = \text{SF}_{V'P'} \cdot |V'| \cdot |P'| = 10^{-4} \cdot 81632 \cdot 333 = 2721
$$

Number of records per blocks
$$
R_{V'P'} = \lfloor \frac{B}{R_{V'P'}} \rfloor = \lfloor \frac{500}{R_{5}} \rfloor = 100
$$

Blocks needed for V'P'
$$
B_{V'P'} = \lceil \frac{|V'P'|}{R_{V'P'}} \rceil = \lceil \frac{2721}{100} \rceil = 28
$$

In [10]:
ndist_region = 30
SF_vp_prime = (1 / ndist_region) * (1 / C_p_prime)
print("Selectivity factor of V'P': {} \n".format(SF_vp_prime))

C_vp_prime = math.floor(SF_vp_prime * C_v_prime * C_p_prime)
print("Cardinality output of V'P': {} \n".format(C_vp_prime))

R_vp_len = 5
B = 500
R_vp_prime = math.floor(B / R_vp_len)
print("V'P' number of records per block : {} \n".format(R_vp_prime))

B_vp_prime = math.ceil(C_vp_prime / R_vp_prime)
print("Blocks needed for V'P': {} \n".format(B_vp_prime))

Selectivity factor of V'P': 0.00010010010010010009 

Cardinality output of V'P': 2721 

V'P' number of records per block : 100 

Blocks needed for V'P': 28 



**Join between W and V'P': WV'P'**

![WV'P'](wvprimepprime2.svg)

Record length for WV'P'
$$
|R_{WV'P'}| = 5
$$

Selectivity Factor for WV'P'
$$
\text{SF} = \frac{1}{|W|} = \frac{1}{5000} = 0.0002
$$

Cardinality Output
$$
|WV'P'| = SF \cdot |W| \cdot |V'P'| = 10^{-4} \cdot 5000 \cdot 2721 = 2721
$$

Number of records per block
$$
R_{WV'P'} = \lfloor \frac{B}{|R_{WV'P'}|} \rfloor = \lfloor \frac{500}{5} \rfloor = 100
$$

Blocks needes for WV'P'
$$
B_{WV'P'} = \lceil \frac{|WV'P'|}{R_{WV'P'}} \rceil = \lceil \frac{2721}{100} \rceil = 28
$$

In [14]:
SF_wv_pr_p_pr = 1 / c_W
print("Selectivity factor of WV'P': {} \n".format(SF_wv_pr_p_pr))

C_wv_pr_p_pr = math.floor(SF_wv_pr_p_pr * c_W * C_vp_prime)
print("Cardinality output of WV'P': {} \n".format(C_wv_pr_p_pr))

R_wv_pr_p_pr_len = 5
B = 500
R_wv_pr_p_pr = math.floor(B / R_wv_pr_p_pr_len)
print("WV'P' number of records per block : {} \n".format(R_wv_pr_p_pr))

B_wv_pr_p_pr = math.ceil(C_wv_pr_p_pr / R_wv_pr_p_pr)
print("Blocks needed for WV'P': {} \n".format(B_wv_pr_p_pr))

Selectivity factor of WV'P': 0.0002 

Cardinality output of WV'P': 2721 

WV'P' number of records per block : 100 

Blocks needed for WV'P': 28 



**PT1/PT2**

**Final result = O**

Record length
$$
|R_O| = 5
$$

Output cardinality
$$
|O| = \text{ndist}(\text{strength}) = 100
$$

Number of records
$$
R_O = \lfloor \frac{B}{|R_O|} \rfloor = \lfloor \frac{500}{5} \rfloor = 100
$$

Blocks needed
$$
B_O = \lceil \frac{|O|}{R_O} \rceil = \lceil \frac{100}{100} \rceil = 1
$$

In [17]:
ndist_strength = 100
C_o = ndist_strength
print("Cardinality output of O: {} \n".format(C_o))

R_o_len = 5
B = 500
R_o = math.floor(B / R_o_len)
print("O number of records per block : {} \n".format(R_o))

B_o = math.ceil(C_o / R_o)
print("Blocks needed for O: {} \n".format(B_o))

Cardinality output of O: 100 

O number of records per block : 100 

Blocks needed for O: 1 



**Map result**

![Phase 2](phase2.svg)

### Phase 3. Cost estimation for each algorithm

Recall:

$$
u = \frac{2}{3} \cdot 2(75) = 100
$$

**AP1/AP2**

**Selection over V: V'**

Recall that for Vintages is clustered by wineId and prodId
Available access paths: No index

$$
\text{cost}_{\text{scan}}(V') = \lceil 1.5 B_{V} \rceil \cdot D = \lceil 1.5 \cdot 5000 \rceil \cdot 1 = 7500
$$

Chosen algorithm: **Scan**

**Selection over P: P'**

Available access paths: B<sup>+</sup> and no index

For a table scan
$$
\text{cost}_{\text{scan}}(P') = \lceil 1.5 B_{P} \rceil \cdot D = \lceil 1.5 \cdot 834 \rceil \cdot 1 = 1251
$$

Tree depth of h for B<sup>+</sup> is:

$$
h = \lceil \log_u |P| \rceil - 1 =  \lceil \log_u |P| \rceil - 1 =  \lceil \log_{100} 10000 \rceil - 1 = 1
$$

For an index of several tuples
$$
\begin{align}
\text{cost}_{B^+}(P') 
& = h \cdot D + \frac{|P'| - 1}{u} \cdot D + |P'| \cdot D \\
& = h \cdot D + \frac{SF_{\text{region = 'Priorat'}} \cdot |P| - 1}{u} \cdot D + SF_{\text{region = 'Priorat'}} \cdot |P| \cdot D \\
& = 1 \cdot 1 + \frac{{^1/_{30}} \cdot 10000 - 1}{100} \cdot D + {^1/_{30}} \cdot 10000 \cdot 1 \\
& = 1 + \frac{332}{100} + 333 \\
& = 337.33
\end{align}
$$

Chosen algorithm: **B<sup>+</sup>**



In [45]:
load = 2/3
d = 75
u = load * (2 * d)
h = math.ceil(math.log(c_P, u)) - 1
D = 1
print("load is: {}\nd is: {}\nu is: {}\nh is: {}\nD is: {}\n".format(load, d, u, h, D))
cost_scan_p = math.ceil(1.5 * B_p) * D
cost_bplus_p = (h * D) + ((C_p_prime / u) * D) + (C_p_prime * D)
print("Cost of scan is: {} \nCost of B+ is: {}".format(cost_scan_p, cost_bplus_p))

load is: 0.6666666666666666
d is: 75
u is: 100.0
h is: 1
D is: 1

Cost of scan is: 1251 
Cost of B+ is: 337.33


**PT1**

**Join over W and V': WV'**

Available algorithms: Block Nested Loops (BML), Row Nested Loops (RML) and Sort-Match (SM)

*Block Nested Loops*

Recall:

$$
M = 4
$$

$\lceil 1.5 B_{W} \rceil < B_{V'}$ we use the commutative property of joins

$$
\begin{align}
\text{cost}_{\text{BML}}(WV') 
& = \lceil 1.5 B_{W} \rceil + \lceil \frac{1.5 B_{W}}{M} \rceil \cdot B_{V'} \\
& = \lceil 1.5 \cdot 500 \rceil + \lceil \frac{1.5\cdot 500}{4} \rceil \cdot 1633 \\
& = 307,754
\end{align}
$$

*Row Nested Loops*

Look for attributes of W

$V'$ does not use extra space any more for being ordered

$$
\begin{align}
\text{cost}_{\text{RML}}(WV') 
& = B_{V'} + |V'| \cdot \left( \lceil \log_u |W| \rceil - 1 + 1 + (\frac{1.5(k-1)}{10} \right) \\
& = 1633 + 81,632 \cdot \left( \lceil \log_{100} 5000 \rceil - 1 + 1 \right) \\
& = 164,887
\end{align}
$$

<span style='color:red'>Note: This wasn't explained. Maybe $k = 1$ but needs confirmation.</span>

*Sort-Match*

$W$ is ordered by `wineID`, $V'$ is still ordered y `wineID` and `prodID`.

$$
\text{cost}_{\text{SM}}(WV') = \lceil 1.5 B_{W} \rceil + B_{V'} = \lceil 1.5 \cdot 500 \rceil + 1633 = 2383
$$

Chosen algorithm: **Sort-Match**

**Join between WV' and P': WV'P'**

*Block Nested Loops*

$B_{P'} < B_{WV'}$ we use the commutative property of joins

$$
\begin{align}
\text{cost}_{\text{BML}}(WV'P') 
& = B_{P'} + \lceil \frac{B_{P'}}{M} \rceil \cdot B_{WV'} \\
& = 4 + \lceil \frac{4}{4} \rceil \cdot 1633 \\
& = 1637
\end{align}
$$

<span style='color:red'>Note: It isn't explained why BML is analyzed but not RML.</span>

*Sort-Match*

Neither WV’ nor P’ are ordered by `prodID`

$$
\begin{align}
\text{cost}_{\text{SM}}(WV'P') 
&= 2 B_{WV'} \cdot \lceil \log_2 B_{WV'} \rceil + 2 B_{P'} \cdot \lceil \log_2 B_{P'} \rceil + B_{WV'} + B_{P'}\\
&= 2 \cdot 1633 \cdot \lceil \log_2 1633 \rceil + 2 \cdot 4 \cdot \lceil \log_2 4 \rceil + 1633 + 4 \\
&= 37,579
\end{align}
$$

Chosen algorithm: **Block Nested Loop**

In [69]:
print("B_p' is {}\nB_wv' is {}".format(B_p_prime, B_wv_prime))

B_p' is 4
B_wv' is 1633


In [72]:
(2 * B_wv_prime * math.ceil(math.log(B_wv_prime, 2))) + (2 * B_p_prime * math.ceil(math.log(B_p_prime, 2))) + B_wv_prime + B_p_prime

37579

**PT2**

**Join between V' and P': V'P'

Available algorithms: BNL and SM.

*Block Nested Loops*

$B_{P'} < B_{V'}$ we use the commutative property of joins

$$
\begin{align}
\text{cost}_{\text{BML}}(V'P') 
& = B_{P'} + \lceil \frac{B_{P'}}{M} \rceil \cdot B_{V'} \\
& = 4 + \lceil \frac{4}{4} \rceil \cdot 1633 \\
& = 1637
\end{align}
$$

*Sort-Match*

Neither V’ nor P’ are ordered by `prodID`

$$
\begin{align}
\text{cost}_{\text{SM}}(V'P') 
&= 2 B_{V'} \cdot \lceil \log_2 B_{V'} \rceil + 2 B_{P'} \cdot \lceil \log_2 B_{P'} \rceil + B_{V'} + B_{P'}\\
&= 2 \cdot 1633 \cdot \lceil \log_2 1633 \rceil + 2 \cdot 4 \cdot \lceil \log_2 4 \rceil + 1633 + 4 \\
&= 37,579
\end{align}
$$

Chosen algorithm: **Block Nested Loop**

In [73]:
print("B_p' is {}\nB_v' is {}".format(B_p_prime, B_v_prime))

B_p' is 4
B_v' is 1633


**Join between W and V'P': WV'P'**

Available algorithms: Block Nested Loops (BML), Row Nested Loops (RML) and Sort-Match (SM)

*Block Nested Loops*

$B_{V'P'} < \lceil 1.5 B_{W} \rceil$ we use the commutative property of joins

$$
\begin{align}
\text{cost}_{\text{BML}}(WV'P') 
& = B_{V'P'} + \lceil \frac{B_{V'P'}}{M} \rceil \cdot \lceil 1.5 B_{W} \rceil \\
& = 28 + \lceil \frac{28}{4} \rceil \cdot \lceil 750 \rceil \\
& = 5278
\end{align}
$$

*Row Nested Loops*

Look for attributes of W

$$
\begin{align}
\text{cost}_{\text{RML}}(WV'P') 
& = B_{V'P'} + |V'P'| \cdot \left( \lceil \log_u |W| \rceil - 1 + 1 + (\frac{1.5(k-1)}{10} \right) \\
& = 28 + 2721 \cdot \left( \lceil \log_{100} 5000 \rceil - 1 + 1 \right) \\
& = 5470
\end{align}
$$

*Sort-Match*

W is ordered by `wineID`, V'P' is not sorted by `wineID`

$$
\begin{align}
\text{cost}_{\text{SM}}(WV'P') 
&= 2 B_{V'P'} \cdot \lceil \log_2 B_{V'P'} \rceil + \lceil 1.5 B_{W} \rceil + B_{V'P'} \\
&= 2 \cdot 28 \cdot \lceil \log_2 28 \rceil + \lceil 1.5 \cdot 500 \rceil + 28 \\
&= 1058
\end{align}
$$

Chosen algorithm: **Sort-Match**

In [82]:
print("B_v'p' is {}\n1.5*B_w is {}\n|V'P'| is {}".format(B_vp_prime, math.ceil(1.5*B_w), C_vp_prime))

B_v'p' is 28
1.5*B_w is 750
|V'P'| is 2721


In [77]:
28 + math.ceil(28/4) * 750

5278

In [83]:
28+(2721*(math.ceil(math.log(5000, 100)) - 1 + 1))

5060.448690899154

In [99]:
Cost_v_prime = 1633 + 7500
Cost_p_prime = 4 + 337
Cost_wv = 1633 + 2383
Cost_vp = 28 + 1637
Cost_wvp_pt1 = 28 + 1637
Cost_wvp_pt2 = 28 + 1058
Cost_o = 1 + 252
Cost_pt1 = Cost_v_prime + Cost_p_prime + Cost_wv + Cost_wvp_pt1 + Cost_o
Cost_pt2 = Cost_v_prime + Cost_p_prime + Cost_vp + Cost_wvp_pt2 + Cost_o
print("Total cost of:\nPT1: {}\nPT2: {}".format(Cost_pt1, Cost_pt2))

Total cost of:
PT1: 15408
PT2: 12478


**Map result**

<span style="color:red">Output algorithm is Merge Sort but it's not explained, nor its cost calculation</span>

![Phase 3](phase3.svg)

### Phase 4. Choose the best option

**PT2**